In [80]:
#!pip install checklist

In [81]:
#!pip install simpletransformers

In [82]:
import numpy as np
import checklist
import pandas as pd
import spacy
from checklist.editor import Editor
from checklist.perturb import Perturb
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,average_precision_score

In [83]:
data = pd.read_csv('/content/data/olid-subset-diagnostic-tests.csv')
data.head(3)

,id,text,labels
0,89200,@USER @USER Who the hell does he think he is?,1
1,71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,55633,"#OrrinHatch I can’t believe this sexist , clue...",1


In [84]:
nlp = spacy.load('en_core_web_sm')

In [85]:
perturbed_data = list(nlp.pipe(data['text']))

In [86]:
perturbed_data[:3]

[@USER @USER Who the hell does he think he is?,
 #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL,
 #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!]

In [87]:
ret = Perturb.perturb(data['text'], Perturb.add_typos)
ret.data[:3][1]

['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#BREAKING. #Greece: Molotov cocktails lfy after protest honouring killed antifa arti... URL via @USER URL']

In [88]:
typo_data = []
for i in range(len(ret.data)):
  typo_data.append(ret.data[i][1]) # tweet with typos added

typo_data[:3]

['@USER @USER Who the hell does he think h eis?',
 '#BREAKING. #Greece: Molotov cocktails lfy after protest honouring killed antifa arti... URL via @USER URL',
 '#OrrinHatch I cna’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!']

In [89]:
data_typo = pd.DataFrame({'text':typo_data})
data_typo.head()

,text
0,@USER @USER Who the hell does he think h eis?
1,#BREAKING. #Greece: Molotov cocktails lfy afte...
2,"#OrrinHatch I cna’t believe this sexist , clue..."
3,@USER @USER I'll use that one the next time im...
4,0-1 lost my acca on the first fcuking fight cba


In [90]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.overwrite_output_dir= True
model_args.output_dir = '/content/drive/MyDrive/NLP/error_analyse_model'


model = ClassificationModel("bert", 'bert-base-cased', args=model_args,use_cuda=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [91]:
train_data = data[['text','labels']]
model.train_model(train_data)

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/13 [00:00<?, ?it/s]

(13, 0.6986603415929354)

In [92]:
predictions, raw_outputs = model.predict(list(data_typo['text']))


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [93]:
predicted_typo_labels = pd.DataFrame({'text':data_typo.text, 'labels': predictions})
predicted_typo_labels.head()

,text,labels
0,@USER @USER Who the hell does he think h eis?,1
1,#BREAKING. #Greece: Molotov cocktails lfy afte...,0
2,"#OrrinHatch I cna’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 lost my acca on the first fcuking fight cba,1


In [94]:
result, model_outputs, wrong_predictions = model.eval_model(predicted_typo_labels, report = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [95]:
print(result['report'])

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        33
         1.0       1.00      1.00      1.00        67

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [96]:
print(model_outputs[:5])

[[-0.67061532 -0.23858932]
 [-0.32077825 -0.37710556]
 [-0.46237618 -0.14922763]
 [-0.65570009 -0.2303382 ]
 [-0.54513955 -0.3187336 ]]


In [97]:
print("Confusion Matrix : ")
print(confusion_matrix(data['labels'], predictions))

Confusion Matrix : 
[[33  0]
 [ 0 67]]


### Provide 3 examples when the model failed to assign the correct label after perturbation.

In [103]:
print(data['labels'][:10])

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: labels, dtype: int64


In [104]:
print(predicted_typo_labels['labels'][:10])

0    1
1    0
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: labels, dtype: int64


In [106]:
failed_texts=[]
failed_labels=[]
for i in range(len(data)):
  if predicted_typo_labels['labels'][i] != data['labels'][i]:
    failed_texts.append(predicted_typo_labels['text'][i])
    failed_labels.append(predicted_typo_labels['labels'][i])

In [109]:
failed_examples = pd.DataFrame({'text':failed_texts, 'labels': failed_labels})
failed_examples

,text,labels
0,#BREAKING. #Greece: Molotov cocktails lfy afte...,0
1,"#BCOG @USER she is cool, confident, cocky, and...",0
2,#Nigeria #Naija #9ja 'You are the most incompe...,0
3,#Cuckservative rTaitors Are Worse Than Fortnit...,0
4,#Conservatives Govt ' @USER made my life hell'...,0
5,#FortniteBattleRoyale #XboxShare @USER Pleas...,0
6,"#NEWS Jeff Sessions: If you want more death,' ...",0
7,". JEFF SESSIONS: ‘LISTEN TO THE ACLU, ANTIFA, ...",0
8,#SandraOh is dressed like the queen she is ton...,1
9,#StayOnYourPath #HealthQuote If you are wishin...,1


# 6. Negation *(4.5 points)* 

In [112]:
perturbed_data = list(nlp.pipe(data['text']))

In [114]:
ret = Perturb.perturb(perturbed_data, Perturb.add_negation) # negated data
ret.data[:5]

[['@USER @USER Who the hell does he think he is?',
  "@USER @USER Who the hell doesn't he think he is?"],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL"],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes.

In [117]:
negated_text = []
for i in range(len(ret.data)):
  negated_text.append(ret.data[i][1])

negated_text[:5]

["@USER @USER Who the hell doesn't he think he is?",
 "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL",
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can't choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 "0-1 didn't lose my acca on the first fucking fight cba"]

In [118]:
neg_predictions, neg_raw_outputs = model.predict(negated_text)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [120]:
predicted_neg_labels = pd.DataFrame({'text':negated_text, 'labels': neg_predictions})
predicted_neg_labels.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,0
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 didn't lose my acca on the first fucking f...,1


In [121]:
result, model_outputs, wrong_predictions = model.eval_model(predicted_neg_labels, neg_report = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [122]:
print(result['neg_report'])

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        27
         1.0       1.00      1.00      1.00        73

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [123]:
print("Confusion Matrix : ")
print(confusion_matrix(data['labels'], predictions))

Confusion Matrix : 
[[25 25]
 [ 8 42]]


### Check the first 10 negated messages. For which of these negated messages should the label be flipped, in your opinion?

In [124]:
predicted_neg_labels['text'][:10]

0     @USER @USER Who the hell doesn't he think he is?
1    #BREAKING. #Greece: Molotov cocktails don't fl...
2    #OrrinHatch I can’t believe this sexist , clue...
3    @USER @USER I'll use that one the next time im...
4    0-1 didn't lose my acca on the first fucking f...
5    #Bakersfield is not why we need gun control! S...
6    #Christian #America – If we go by #Trump’s exa...
7    @USER @USER @USER She is not the most disingen...
8    #Democrats #Liberals you are being #threatened...
9    699. Just didn't want to tell you you should H...
Name: text, dtype: object

In [125]:
failed_neg_texts=[]
failed_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels['labels'][i] != data['labels'][i]:
    failed_neg_texts.append(predicted_neg_labels['text'][i])
    failed_neg_labels.append(predicted_neg_labels['labels'][i])

In [126]:
failed_neg_examples = pd.DataFrame({'text':failed_neg_texts, 'labels': failed_neg_labels})
failed_neg_examples

,text,labels
0,#BREAKING. #Greece: Molotov cocktails don't fl...,0
1,#Nigeria #Naija #9ja 'You are not the most inc...,0
2,#Cuckservative Traitors Are not Worse Than For...,0
3,#Conservatives Govt ' @USER didn't make my lif...,0
4,#FortniteBattleRoyale #XboxShare @USER Pleas...,0
5,"#NEWS Jeff Sessions: If you want more death, '...",0
6,". JEFF doesn't session : ‘LISTEN TO THE ACLU, ...",0
7,#SandraOh is not dressed like the queen she is...,1
8,#StayOnYourPath #HealthQuote If you are wishin...,1
9,#SomethingIsAboutToHappen And here doesn't go ...,1


In [127]:
success_neg_texts=[]
success_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels['labels'][i] == data['labels'][i]:
    success_neg_texts.append(predicted_neg_labels['text'][i])
    success_neg_labels.append(predicted_neg_labels['labels'][i])

In [128]:
successful_neg_examples = pd.DataFrame({'text':success_neg_texts, 'labels': success_neg_labels})
successful_neg_examples

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,"#OrrinHatch I can’t believe this sexist , clue...",1
2,@USER @USER I'll use that one the next time im...,1
3,0-1 didn't lose my acca on the first fucking f...,1
4,#Bakersfield is not why we need gun control! S...,1
...,...,...
58,#Conservatives are always not revolting URL,0
59,@USER She Is not A ChiCom Spy..... #ClubGITMO,0
60,#Repost @USER ・・・ We can’t forget to give prop...,0
61,#Canada - EXCLUSIVE: #Trudeau #Liberals don't ...,0


## 7. Creating negated examples